In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat 


data = loadmat('ex4data1.mat')
data # Dictionary object 

{'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [2]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [3]:
X = data['X'] # Converts X to numpy ndarray
y = data['y'] # Converts y to numpy ndarray

print(X.shape)
print(y.shape)

(5000, 400)
(5000, 1)


In [4]:
m = X.shape[0]

In [5]:
# Adding column of ones to data matrix X
X = np.c_[np.ones((m,1)), X]
X.shape

(5000, 401)

In [6]:
# Loading weights
weights = loadmat('ex4weights.mat')
weights.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Theta1', 'Theta2'])

In [7]:
theta1, theta2 = weights['Theta1'], weights['Theta2']
print('theta1 : ', theta1.shape)
print('theta2 : ', theta2.shape)
# Unrolling theta1 and theta2 vector
nn_params = np.r_[theta1.ravel(), theta2.ravel()]
print('nn_params : ', nn_params.shape)

theta1 :  (25, 401)
theta2 :  (10, 26)
nn_params :  (10285,)


In nnCostFunction the formal parameter nn_params is always the array of unrolled weights that’s passed to it as an argument from the calling function (which is sometimes ex4 and sometimes fmincg). In ex4 the actual argument nn_params is the unrolled vector from the Theta weights loaded from the ‘ex4weights.mat’ file, and initial_nn_params is the unrolled vector from the Theta weights initialized by randInitializeWeights.

ex4 calls nnCostFunction with nn_params directly a few times to check the forward propagation calculations, and calls fmincg with initial_nn_params to kick off its version of gradient descent. fmincg then calls nnCostFunction indirectly through the “short hand” costFunction to compute the cost and gradients and then fmincg updates the weights and repeats the optimization process until either there’s convergence or else the maximum number of iterations is reached.

The optimization engine fmincg we use is a general purpose function and doesn’t know anything about neural networks. It just minimizes a cost based on an array of (unrolled) parameters and their corresponding gradients. It passes those parameters to nnCostFunction which does know about neural networks and reshapes the general parameter list back into Θ matrices to compute the cost and gradients.

### Neural Networks - Feed Forward and Back Prop

Input layer size = 400 (20 x 20)

Hidden layer size = 25

Number of labels = 10

In [8]:
# Define sigmoid function
def sigmoid(z):
    return (1 / (1 + np.exp(-z))) 

In [9]:
# Sigmoid derivative
def sigmoid_derivative(z):
    s = sigmoid(z)
    return (s * (1 - s))

In [50]:
def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambdaa):
    
    # When comparing with octave code, Note that Python uses indexing from 0
    # + 1 as we added colum of ones previously
    theta1 = nn_params[0 : (hidden_layer_size * (input_layer_size + 1))].reshape(hidden_layer_size, input_layer_size + 1)
    # theta1 is rolled into a matrix of shape (25, 401) containg 10025 elements
    
    theta2 = nn_params[(hidden_layer_size * (input_layer_size + 1)) : ].reshape(num_labels, hidden_layer_size + 1)
    # theta2 is rolled into a matrix of shape (10, 26) containg 260 elements
    
    # PART - 1 FORWARD PROP AND COST
    a1 = X
    
    # layer 1 to layer 2
    z2 = np.dot(a1, theta1.T) # (5000, 401) * (25, 401).T
    a2 = sigmoid(z2)
    
    # layer 2 to layer 3
    a2 = np.c_[np.ones((m,1)), a2] # (5000,1) column-wise concatenation (5000, 25) = (5000, 26)
    z3 = np.dot(a2, theta2.T) # (5000,10)
    a3 = sigmoid(z3)
    
    # Converting 'y' vector to matrix 'y_mat'
    #y_mat = pd.get_dummies(classes.ravel()).as_matrix()
    
    # Regularized cost
    reg_term = np.sum(np.sum(theta1[:, 1:] ** 2)) + np.sum(np.sum(theta2[:, 1:] ** 2)) 
    J = (-1/m) * np.sum(y * np.log(a3) + (1 - y) * np.log(1 - a3)) + (lambdaa/(2*m)) * reg_term
    
    # PART - 2 BACK PROP TO COMPUTE THE GRADIENT Theta1_grad and Theta2_grad
    tridelta1 = 0
    tridelta2 = 0
    
    # Error for output layer is output - predicted output
    d3 = a3 - y # (5000,10)
    
    # Error for hidden layer; from the formula used in lectures
    d2 = np.dot(d3, theta2[:, 1:]) * sigmoid_derivative(z2) # (5000, 10) * (10, 25)
    
    tridelta1 = tridelta1 + np.dot(d2.T, a1) # (25,401) = theta1.shape
    tridelta2 = tridelta2 + np.dot(d3.T, a2) # (10,26) = theta2.shape
    
    # Regularized gradient
    # Set first column of theta1 and theta2 to zero ie; ignore the bias term
    Theta1 = np.c_[np.zeros((theta1.shape[0], 1)), theta1[:, 1:]]
    Theta2 = np.c_[np.zeros((theta2.shape[0], 1)), theta2[:, 1:]]
    
    theta1_grad = (1/m) * (tridelta1 + lambdaa * theta1)
    theta2_grad = (1/m) * (tridelta2 + lambdaa * theta2)
    
    return (J, theta1_grad, theta2_grad)

We're also going to need to one-hot encode our labels. One-hot encoding turns a class label n (out of k classes) into a vector of length k where index n is "hot" (1) while the rest are zero. Scikit-learn has a built in utility we can use for this.

In [51]:
from sklearn.preprocessing import OneHotEncoder

# Returns an array; would return sparse matrix if sparse = True
encoder = OneHotEncoder(sparse = False)

y_OneHot = encoder.fit_transform(y)

In [53]:
nnCostFunction(nn_params, 400, 25, 10, X, y_OneHot, 1)

(0.38376985909092365,
 array([[ 5.73587986e-05, -2.11248326e-12,  4.38829369e-13, ...,
          7.09042553e-09,  1.84706139e-09,  5.60928898e-13],
        [ 7.42035851e-05,  1.53233736e-12, -1.95174738e-12, ...,
          2.10747891e-08, -8.61281252e-11,  7.08845709e-13],
        [-1.69362396e-04, -1.75530893e-12,  1.63207553e-12, ...,
          4.63501184e-08,  9.48509834e-10, -1.50133620e-12],
        ...,
        [ 2.94128026e-05, -1.77854412e-12, -1.96393620e-12, ...,
         -9.34100149e-09,  1.29689159e-09,  1.80499812e-12],
        [ 1.50102796e-04,  6.10356747e-14,  5.12122016e-13, ...,
          3.33797620e-07, -3.66032512e-08,  7.67523996e-13],
        [-1.33561977e-04,  1.77175372e-12, -1.31503028e-13, ...,
          4.69418663e-09,  2.83929031e-09,  1.75890906e-12]]),
 array([[ 4.76536940e-04,  5.08457277e-04,  7.84221985e-05,
          1.01449847e-03,  5.20245821e-04,  9.39490340e-04,
         -4.65577532e-05, -4.26660299e-04, -8.07731636e-04,
         -7.71028631e-06,  